In [ ]:
# coding=utf-8
# Copyright 2023 Frank Latos AC8P
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#
# Much appreciation to the Pymoo project for providing the optimization framework used herein:
#
# pymoo: Multi-objective Optimization in Python
# https://github.com/anyoptimization/pymoo
# https://pymoo.org/index.html
#


# A Class for Representing Yagis and Other Antennas:<br>Part 3, Setting Properties

You can define the geometry of your antenna by setting properties; here's a fuller discussion of the details.


The ```SolidAntenna``` class that you base your design on already has some default properties set; these are global properties that affect the overall antenna and all of its elements. As of this writing they look someting like:
```python
self.properties = {'tubing_dia':in2m([3/4, 5/8, 1/2]), 'section_lens':in2m([24,18]), 
                    'color':['lightblue','lightsteelblue','lightgray','whitesmoke'], 'nsides':9,
                    'boom':True, 'boomcolor':'steelblue', 'boomext':in2m(4), 'boomy':in2m(3), 'boomz':in2m(2),
                    'rotate':[], 'translate':(0,0,0)}
```
You can add or modify these when you create a new object:
```python
my_antenna = SolidAntenna(properties={'translate':(0,0,20), 'display_mirror':True, 'seg_per_m':5}, n_elem=3)
```
or at any later time like this:
```python
my_antenna.properties={'translate':(0,0,20), 'display_mirror':True, 'seg_per_m':5}, n_elem=3)
```


